# Natural Language Processing

We starten weer met het laden van de benodigde package.

In [ ]:
import pandas as pd

En het inladen van onze gefilterde dataset.

In [ ]:
data = pd.read_csv('data/fiets.csv', index_col =0)

In [ ]:
data.head(2)

In [ ]:
len(data)

## Woord frequenties

Als eerste, meeste simpele stap van tekst analyse, kunnen we kijken naar de meest voorkomende woorden in de artikelen. Onderstaande functie is geschreven om dit te doen. 

Maar voordat we deze functie kunnen gebruiken moeten we de artikelen van de kolom `content` omzetten naar een lijst met woorden per artikel. 
Dit doen we met de `split` functie. Omdat we voor het tellen geen rekening willen houden met hoofdletters, zetten we alle letters om naar kleine letters. Dit doen we met de functie `lower`. 

In [ ]:
data['content_list'] = data.content.apply(lambda x: x.lower().split())

In [ ]:
def word_counter(dataframe_column):
    full_list = []
    for elemnt in dataframe_column:
        full_list += elemnt
    
    values_count = pd.Series(full_list).value_counts()
    return values_count

Nu we de kolom `content_list` hebben, kunen we de `word_counter` functie uitproberen. 

In [ ]:
word_counter(data['content_list'])

Zoals je ziet zijn de eerste woorden allemaal woorden die inhoudelijk niet veel zeggen over de artikelen. De meest voorkomende woorden in teksten zijn vaak lidwoorden, voegwoorden, voorzetsels en dergelijke. Dit zijn woorden die vaak weinig informatie toevoegen over de inhoud van teksten. In Natural Language Processing noemen we deze woorden `stopwoorden`.  Afhankelijk van de analyse en de dataset kunnen de gebruikte stopwoorden veranderen. Voor Nederlands zijn veelgebruikte stopwoorden de lidwoorden, voegwoorden en voornaamwoorden. Ook artefacten uit de tekst kunnen worden toegevoegd. 

In onze datafolder staat een stopwoordenlijst klaar die we kunnen gebruiken voor het uitfilteren van deze woorden. 

In [ ]:
with open('data/stopwords-nl.txt', 'r') as f:
    stopwords = f.read().split("\n")

En hieronder staat een nieuwe functie, die deze woorden meeneemt als filter. 

In [ ]:
def word_counter_stopword(dataframe_column):
    full_list = []
    for elemnt in dataframe_column:
        full_list += elemnt
        
    full_list = [i for i in full_list if i not in stopwords]
    
    values_count = pd.Series(full_list).value_counts()
    return values_count

Nu kunnen we onze nieuwe functie `word_counter_stopword` draaien over de kolom `content_list`. 

In [ ]:
word_counter_stopword(data['content_list'])

Dit ziet er al een stuk beter uit. Met behulp van de `head` functie kunnen we de top 10 tonen. Zoals je ziet is het nog niet helemaal perfect, want het meest voorkomende teken is een streepje, en er staat nog een komma tussen. Normaliter schoon je de tekst dus nog wat verder op voordat je er mee verder gaat. Omdat dit puur ter demonstratie is slaan we deze stap nu over. 

In [ ]:
word_counter_stopword(data['content_list']).head(10)

# Spacy

Naast een naieve wordcount, zijn er ook andere opties mogelijk. Voor het vervolg van deze workshop gaan we gebruik maken van de Spacy package. Deze package wordt veel gebruikt wordt voor Natural Language Processing. Een andere veelgebruikte package is `NLTK`. 

Voor het vervolg van dit Notebook hebben we aantal packages nodig. Dus we starten met het importeren van deze packages. 

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import pickle

Als je Spacy vraagt om Natural Language Processing uit te voeren, worden voor elke input verschillende Natural Language Processing taken uitgevoerd, waaronder het herkennnen van entities en woordsoorten. Deze informatie wordt opgeslagen als een `doc` item. Het draaien van deze code duurt enige tijd. We hebben daarom het model alvast voorbereid en klaargezet in de map `data`. De code voor het draaien van een model staat hieronder als voorbeeld aangegeven. Dit kun je gebruiken als je je eigen  model wilt draaien op bijvoorbeeld een andere dataset. 

```
nlp = spacy.load("nl_core_news_sm")
data = data.dropna(subset=['content'])

def process_text(text):
   return nlp(text)
   
data["doc"] = data["content"].apply(process_text)

with open('data/fiets_nlp.pkl', 'wb') as f:
    pickle.dump(data, f)

```

Voor het vervolg van dit Notebook laden we het model dat in de `data` map staat. Dit doen we met de volgende code:

In [ ]:
fiets_nlp = pd.read_pickle("data/fiets_nlp.pkl")

In [ ]:
fiets_nlp.head(2)

# Named Entity Recognition

Spacy slaat informatie over de gevonden named entities op in het doc item. Deze informatie staat opgeslagen in de kolom `doc` en kan daar ook uitgehaald worden.

Spacy kent de volgende Named entities:

* PERSON:      People, including fictional.
* NORP:        Nationalities or religious or political groups.
* FAC:         Buildings, airports, highways, bridges, etc.
* ORG:         Companies, agencies, institutions, etc.
* GPE:         Countries, cities, states.
* LOC:         Non-GPE locations, mountain ranges, bodies of water.
* PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
* EVENT:       Named hurricanes, battles, wars, sports events, etc.
* WORK_OF_ART: Titles of books, songs, etc.
* LAW:         Named documents made into laws.
* LANGUAGE:    Any named language.
* DATE:        Absolute or relative dates or periods.
* TIME:        Times smaller than a day.
* PERCENT:     Percentage, including ”%“.
* MONEY:       Monetary values, including unit.
* QUANTITY:    Measurements, as of weight or distance.
* ORDINAL:     “first”, “second”, etc.
* CARDINAL:    Numerals that do not fall under another type.

Je kan ook de spacy module vragen voor uitleg over een bepaalde Named Entity:

In [ ]:
spacy.explain('GPE')

Je kan per artikel de gevonden Named Entities tonen. Deze worden dan met verschillende kleuren in de tekst weergegeven.

Eerst maken we een variabele `doc` aan, waarin we de inhoud van de kolom `doc` voor één artikel stoppen. Dit doen we met de blokhaken `[2]`. In dit geval roepen we het derde artikel aan (Python telling begint bij 0).

In [ ]:
doc = fiets_nlp['doc'].to_list()[2]

Vervolgens kunnen we met de functie `displacy.render()` dit artikel inclusief de bijbehorende Named Entities tonen. 

In [ ]:
displacy.render(doc, style = "ent")

### **Oefening**
We hebben net het derde artikel bekeken. Kies nu zelf een ander artikel om te bekijken. 
Maak eerst een variabel aan voor dit artikel, en roep dan de functie `displacy.render()` aan. 

In [ ]:
## Maak een nieuwe variabele met de content van één artikel
## Voorbeeld syntax: variabele_naam = fiets_nlp['doc'].to_list[nummer van het artikel]

In [ ]:
## Gebruik displacy.render() om het artikel te tonen
## Voorbeeld syntax: displacy.render(variable_naam, style = "ent")

Naast de Named Entities op deze manier tonen, kun je er ook verdere analyses mee doen. Je kan ze bijvoorbeeld per category opslaan als losse kolommen in je dataframe en deze kolommen gebruiken om de meest voorkomende entities per categorie te tonen. 

Hieronder staat een functie waarmee je Named Entities van een bepaalde soort uit de `doc` kolom kan halen. Deze functie kan je gebruiken om de gevonden entities als een lijst op te slaan in een nieuwe kolom in het dataframe. 

In [ ]:
def get_ner(doc, entity):
    return [ent.text for ent in doc.ents if ent.label_ == entity]

In het onderstaande voorbeeld wordt de category `GPE` uit het`doc` item gehaald. Deze informatie wordt vervolgens opgeslagen in de kolom `GPE`. 

In [ ]:
fiets_nlp['GPE'] = fiets_nlp['doc'].apply(lambda x: get_ner(x, 'GPE'))

Vervolgens kun je deze nieuwe kolom weer gebruiken om een top 10 van de meest voorkomende GPE locaties uit de dataset te halen. 

In [ ]:
word_counter(fiets_nlp['GPE']).head(10)

### **Oefening** 
Maak nu zelf een kolom aan voor een andere Named Entity naar keuze, en bekijk de top 10. Kies een entity uit de lijst hierboven. 

In [ ]:
## Stap 1: maak een nieuwe kolom aan voor de gekozen entity
## Voorbeeld syntax: fiets_nlp['naam nieuwe kolom'] = fiets_nlp['doc'].apply(lambda x: get_ner(x, 'naam entity'))

In [ ]:
## Stap 2: Toon de tien meest voorkomende woorden van je gekozen category.
## Word counter: word_counter(fiets_nlp['nieuwe kolom']).head(10)

### **Bonus oefening**
Doe hetzelfde voor enkele andere Named Entities. Kun je ook een top 15 of 25 maken?

In [ ]:
## Schrijf hier de code voor de bonus oefening

Hierboven hebben we telkens de top 10 van een Named Entity op de hele dataset getoond. Je kunt de dataset ook splitsen, en dan de top 10 van de verschillende subframes vergelijken. 

We geven hieronder een voorbeeld voor de Named Entity `GPE` en we splitsen de dataframe op basis van de kolom `spatial`. 

We beginnen  met het maken van twee subdataframes: `landelijk_df` en `lokaal_df`.

In [ ]:
landelijk_df = fiets_nlp[fiets_nlp['spatial'] == 'Landelijk']

In [ ]:
regionaal_df = fiets_nlp[fiets_nlp['spatial'] == 'Regionaal/lokaal']

Nu kunnen we deze subframes gebruiken voor een top 10 per frame, waarna we deze kunnen vergelijken met elkaar. 

In [ ]:
word_counter(landelijk_df['GPE']).head(10)

In [ ]:
word_counter(regionaal_df['GPE']).head(10)

### **Oefening**
Gebruik de subframes `landelijk_df` en `regionaal_df` om de meest voorkomenden woorden van jouw eigen hierboven gekozen entity te vergelijken. Gebruik hiervoor de kolom die bij de oefening hierboven zelf hebt aangemaakt. 

In [ ]:
## Voorbeel syntax: word_counter(landelijk_df['kolomnaam eigen entity']).head(10)

In [ ]:
## Voorbeel syntax: word_counter(regionaal_df['kolomnaam eigen entity']).head(10)